In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

plt.rcParams['figure.figsize'] = (10, 6)

# Read in train & val data

In [ ]:
train_df = sqlContext.read.load("data/train_100_grayscale.parquet")
val_df = sqlContext.read.load("data/val_100_grayscale.parquet")
train_df, val_df

In [ ]:
tc = train_df.count()
vc = val_df.count()
tc, vc  # 100

In [ ]:
train_df.select("tumor_score").groupBy("tumor_score").count().show()
val_df.select("tumor_score").groupBy("tumor_score").count().show()

In [ ]:
C = 1
SIZE = train_df.first().sample.toArray().shape[0]
SIZE

# Test out local iterator of RDD

In [ ]:
iterator = train_df.rdd.toLocalIterator()

In [ ]:
b = next(iterator)

In [ ]:
type(b)

In [ ]:
len(b)

In [ ]:
t = np.array([b.sample.values, b.sample.values])
u = np.array([b.tumor_score, b.tumor_score]).astype(np.int8)
t.shape, t.dtype, u.shape, u.dtype

# Create batch generator

In [ ]:
iterator = train_df.rdd.toLocalIterator()

In [ ]:
# Create generator that cycles through the data and yields a batch at a time,
# reinitializing the iterator as needed to continue yielding batches.
# Look at TensorFlow MNIST batch generator.
# Should be able to pass in

In [ ]:
# pseudocode
def gen_batch(rdd, batch_size=50):
  iterator = rdd.toLocalIterator()
  while True:
    features = []
    labels = []
    for i in range(batch_size):
      # Generate batch
      try:
        row = next(iterator)
      except StopIteration:
        # Restart iterator
        iterator = rdd.toLocalIterator()
        row = next(iterator)
      features.append(row.sample.values)
      labels.append(row.tumor_score)
    x_batch = np.array(features).astype(np.float32)
    y_batch = np.array(labels).astype(np.int32)
    yield x_batch, y_batch

In [ ]:
generator = gen_batch(train_df.rdd, 32)

In [ ]:
x, y = next(generator)
x, y

In [ ]:
x.shape, x.dtype, y.shape, y.dtype

# Get validation data

In [ ]:
val_df

In [ ]:
rows = val_df.take(3)  #.collect()

In [ ]:
x = np.array([row.sample.toArray().astype(np.float32) for row in rows])
y = np.array([row.tumor_score for row in rows])

In [ ]:
x.dtype, x.shape, y.dtype, y.shape

In [ ]:
def extract_data(df):
  rows = df.collect()
  x = np.array([row.sample.toArray().astype(np.float32) for row in rows])
  y = np.array([row.tumor_score for row in rows])
  return x, y

# Create softmax model graph

# Create convnet model graph
Create network:
  conv1 -> relu1 -> pool1 -> conv2 -> relu2 -> pool2 -> conv3 -> relu3 -> pool3 -> affine1 -> relu1 -> affine2 -> softmax

In [ ]:
tf.reset_default_graph()

# Hyperparams & Settings
classes = 3
features = 65536
C = 1  # Number of input channels (dimensionality of input depth)
Hin = 256  # Input height
Win = 256  # Input width
Hf = 3  # conv filter height
Wf = 3  # conv filter width
Hfp = 2  # pool filter height
Wfp = 2  # pool filter width
stride = 1  # conv stride
pstride = 2  # pool stride
pad = 1  # For same dimensions, (Hf - stride) / 2
F1 = 32  # num conv filters in conv1
F2 = 32  # num conv filters in conv2
F3 = 32  # num conv filters in conv3
N1 = 512  # num nodes in affine1
lr = 1e-4 # learning rate

# Inputs
with tf.name_scope("input") as scope:
  x = tf.placeholder(tf.float32, [None, features], name="x")
  x_image = tf.transpose(tf.reshape(x, [-1, C, Hin, Win]), perm=[0,2,3,1])  # shape (N,H,W,C)
  y_ = tf.placeholder(tf.int64, [None, ], name="y_")
  y_one_hot = tf.one_hot(y_-1, classes)  # or use sparse cross entropy
  tf.summary.image("x", x_image)
  tf.summary.histogram("y", y_)

# Conv layer 1: conv1 -> relu1 -> pool1
with tf.name_scope("conv1") as scope:
  W = tf.Variable(tf.random_normal([Hf, Wf, C, F1]) * np.sqrt(2.0/(Hf*Wf*C)), name="W")
  b = tf.Variable(tf.zeros([F1]), name="b")
  conv = tf.nn.conv2d(x_image, W, [1,stride,stride,1], padding="SAME") + b
  relu = tf.nn.relu(conv)
  pool = tf.nn.max_pool(relu, ksize=[1,Hfp,Wfp,1], strides=[1,pstride,pstride,1], padding="SAME")
  tf.summary.image("conv1", tf.transpose(W, [3,0,1,2]), max_outputs=F1)  # transpose to [N,H,W,C]

# Conv layer 2: conv2 -> relu2 -> pool2
with tf.name_scope("conv2") as scope:
  W = tf.Variable(tf.random_normal([Hf, Wf, F1, F2]) * np.sqrt(2.0/(Hf*Wf*F1)), name="W")
  b = tf.Variable(tf.zeros([F2]), name="b")
  conv = tf.nn.conv2d(pool, W, [1,stride,stride,1], padding="SAME") + b
  relu = tf.nn.relu(conv)
  pool = tf.nn.max_pool(relu, ksize=[1,Hfp,Wfp,1], strides=[1,pstride,pstride,1], padding="SAME")

# Conv layer 3: conv3 -> relu3 -> pool3
with tf.name_scope("conv3") as scope:
  W = tf.Variable(tf.random_normal([Hf, Wf, F2, F3]) * np.sqrt(2.0/(Hf*Wf*F2)), name="W")
  b = tf.Variable(tf.zeros([F3]), name="b")
  conv = tf.nn.conv2d(pool, W, [1,stride,stride,1], padding="SAME") + b
  relu = tf.nn.relu(conv)
  pool = tf.nn.max_pool(relu, ksize=[1,Hfp,Wfp,1], strides=[1,pstride,pstride,1], padding="SAME")

# Affine layer 1:  affine1 -> relu1 -> dropout
with tf.name_scope("affine1") as scope:
  D = int(F3*(Hin/2**3)*(Win/2**3))
  W = tf.Variable(tf.random_normal([D,N1]) * np.sqrt(2.0/D), name="W")
  b = tf.Variable(tf.zeros([N1]), name="b")
  affine = tf.matmul(tf.reshape(pool, [-1,D]), W) + b
  relu = tf.nn.relu(affine)
  keep_prob = tf.placeholder(tf.float32, name="keep_prob")
  dropout = tf.nn.dropout(relu, keep_prob)

# Affine layer 2:  affine2 -> softmax
with tf.name_scope("affine2") as scope:
  W = tf.Variable(tf.random_normal([N1,classes]) * np.sqrt(2.0/N1), name="W")
  b = tf.Variable(tf.zeros([classes]), name="b")
  logits = tf.matmul(dropout, W) + b
  probs = tf.nn.softmax(logits)
  tf.summary.histogram("logits", logits)
  tf.summary.histogram("probs", probs)

# Loss
with tf.name_scope("loss") as scope:
  cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, y_one_hot))
  tf.summary.scalar("loss", cross_entropy_loss)

# Train
# train_step = tf.train.GradientDescentOptimizer(lr).minimize(cross_entropy)
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy_loss)

# Eval metrics
with tf.name_scope("eval") as scope:
  correct_pred = tf.equal(tf.argmax(logits,1), tf.argmax(y_one_hot,1))
  accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
  tf.summary.scalar("accuracy", accuracy)

In [ ]:
# Run `tensorboard --logdir=tf_logs --host=localhost --debug --reload_interval 5`
with tf.Session() as sess:
  # Summaries
  log_dir = "tf_logs"
  summary_op = tf.summary.merge_all()
  train_writer = tf.train.SummaryWriter(log_dir + "/train", sess.graph)
  val_writer = tf.train.SummaryWriter(log_dir + "/val")
  
  # Data Gen
  train_generator = gen_batch(train_df.rdd, 64)
  x_val, y_val = extract_data(val_df)
  
  # Train
  sess.run(tf.global_variables_initializer())
  steps = 100
  for i in range(steps):
    xs, ys = next(train_generator)
    _ = sess.run([train_step], feed_dict={x: xs, y_:ys, keep_prob:0.5})
    if i % 10 == 0:
      # train stats
      summary, train_acc = sess.run([summary_op, accuracy], feed_dict={x: xs, y_:ys, keep_prob:0.5})
      train_writer.add_summary(summary, i)
      # val stats
      summary, val_acc = sess.run([summary_op, accuracy], feed_dict={x: x_val, y_:y_val, keep_prob:0.5})
      val_writer.add_summary(summary, i)
      #train_writer.flush()  # To force write -- probably slower (usually asynchronous)
      #val_writer.flush()  # To force write -- probably slower (usually asynchronous)
      print("Iter: {}, \t Train Accuracy: {:.4f}, \t Val Accuracy: {:.4f}".format(i, train_acc, val_acc))
train_writer.flush()  # Make sure everything is written before exiting
val_writer.flush()  # Make sure everything is written before exiting

In [ ]:
# 1. Add TensorBoard summaries and track.
# 2. Plug into larger dataset.
# 3. Run on cluster.
# 4. Explore saving to TFRecord format, then reading from files shared on DFS (gfs).

# Convert data to TFRecords

In [ ]:
from tensorflow.contrib.learn.python.learn.datasets import mnist

data_sets = mnist.read_data_sets("/tmp/data",
                                   dtype=tf.uint8,
                                   reshape=False,
                                   validation_size=5000)

In [ ]:
data_sets

In [ ]:
data_sets.train

In [ ]:
data_sets.train.images.shape, data_sets.train.images.dtype

In [ ]:
data_sets.train.labels.shape, data_sets.train.labels.dtype

In [ ]:
train_df.rdd.glom().map(len).collect()

In [ ]:
train_df.rdd.getNumPartitions()

In [ ]:
train_df.repartition(3).rdd.getNumPartitions()

In [ ]:
train_df.repartition(3).rdd.glom().map(len).collect()

In [ ]:
train_df.foreachPartition?

In [ ]:
im = train_df.first().sample.toArray()

In [ ]:
im.shape

In [ ]:
import os

In [ ]:
if not os.path.exists("/tmp/tf"):
  os.mkdir("/tmp/tf")

In [ ]:
writer1 = tf.python_io.TFRecordWriter("/tmp/tf/testing.tfrecords")

In [ ]:
writer2 = tf.python_io.TFRecordWriter("/tmp/tf/testing.tfrecords")

In [ ]:
writer1.close(), writer2.close()

In [ ]:
def gen_batch(rdd, batch_size=50):
  iterator = rdd.toLocalIterator()
  while True:
    features = []
    labels = []
    for i in range(batch_size):
      # Generate batch
      try:
        row = next(iterator)
      except StopIteration:
        # Restart iterator
        iterator = rdd.toLocalIterator()
        row = next(iterator)
      features.append(row.sample.values)
      labels.append(row.tumor_score)
    x_batch = np.array(features).astype(np.float32)
    y_batch = np.array(labels).astype(np.int32)
    yield x_batch, y_batch

In [ ]:
row = train_df.rdd.take(2)[1]
#row

In [ ]:
row['__INDEX'], row.slide_num, row.tumor_score, row.molecular_score, row.sample.toArray()

In [ ]:
tf.train.FloatList?

In [ ]:
import os
import uuid

def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def convert(rows, folder):
  """Convert data to file of tfrecords."""
  # TODO: A uuid is highly unlikely to overlap, but we implement a truly
  # unique identifier.  We could possibly `mapPartitions` to lists, then
  # `zipWithUniqueID` on each list, then write each entry, using the ID
  # as the filename.
  filename = os.path.join(folder, str(uuid.uuid4()) + ".tfrecords")
  writer = tf.python_io.TFRecordWriter(filename)
  for row in rows:
    example = tf.train.Example(features=tf.train.Features(feature={
          '__INDEX': _int64_feature(row['__INDEX']),
          'slide_num': _int64_feature(row.slide_num),
          'tumor_score': _int64_feature(row.tumor_score),
          'molecular_score': _float_feature(row.molecular_score),
          'sample': _bytes_feature(row.sample.toArray().astype(np.float32).tostring()) # should use the uint8 DataFrame
          # TODO: ENCODE SHAPE
        }))
    writer.write(example.SerializeToString())
  writer.close()

In [ ]:
folder = "data/tf"
if not os.path.exists(folder):
  os.mkdir(folder)
rows = train_df.take(5)
convert(rows, folder)  # side-effect of writing to file

In [ ]:
os.listdir(folder)

In [ ]:
filename = os.path.join(folder, os.listdir(folder)[0])
filename

In [ ]:
# 1. Check outputs for correctness
# 2. write partitions and check output
# 3. Run on cluster

In [ ]:
tf.python_io.tf_record_iterator?

In [ ]:
tf.parse_single_example?

In [ ]:
tf.TFRecordReader.read?

In [ ]:
tf.train.string_input_producer?

In [ ]:
tf.FixedLenFeature?

In [ ]:
tf.reset_default_graph()
filename_queue = tf.train.string_input_producer([filename])

reader = tf.TFRecordReader()
a, b = reader.read(filename_queue)

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
with tf.Session() as sess:
  sess.run(init_op)
  coord = tf.train.Coordinator()
  threads = tf.train.start_queue_runners(sess=sess, coord=coord)
  a1, b1 = sess.run([a, b])
a1 #, b1  # a1 is filename, b1 is the contents of that file

In [ ]:
tf.decode_raw?

In [ ]:
def read_and_decode(filename_queue):
  """Read TFRecords from a file and decode into single examples."""
  reader = tf.TFRecordReader()
  filename, serialized_example = reader.read(filename_queue)
  features = tf.parse_single_example(
      serialized_example,
      features={
          '__INDEX': tf.FixedLenFeature([], tf.int64),
          'slide_num': tf.FixedLenFeature([], tf.int64),
          'tumor_score': tf.FixedLenFeature([], tf.int64),
          'molecular_score': tf.FixedLenFeature([], tf.float32), # default for Python float is float32
          'sample': tf.FixedLenFeature([], tf.string)
          # TODO: DECODE SHAPE
      })
  index = features["__INDEX"]  #tf.cast(features["__INDEX"], tf.int8)
  slide_num = features["slide_num"]  #tf.cast(features["slide_num"], tf.int32)
  tumor_score = features["tumor_score"]
  molecular_score = features["molecular_score"]
  sample = tf.decode_raw(features["sample"], tf.float32)  # should use the uint8 DataFrame
  
  return index, slide_num, tumor_score, molecular_score, sample

## Check that first couple of examples are correct

In [ ]:
tf.reset_default_graph()

filename = os.path.join(folder, os.listdir(folder)[0])
filename_queue = tf.train.string_input_producer([filename])

index, slide_num, tumor_score, molecular_score, sample = read_and_decode(filename_queue)

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

sess = tf.Session()

sess.run(init_op)
#   coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess)  #, coord=coord)

index1, slide_num1, tumor_score1, molecular_score1, sample1 = sess.run([index, slide_num, tumor_score, molecular_score, sample])
index2, slide_num2, tumor_score2, molecular_score2, sample2 = sess.run([index, slide_num, tumor_score, molecular_score, sample])

In [ ]:
index1, slide_num1, tumor_score1, molecular_score1, sample1

In [ ]:
row1 = rows[0]
row1

In [ ]:
np.allclose(row1.sample.toArray(), sample1)

In [ ]:
index2, slide_num2, tumor_score2, molecular_score2, sample2

In [ ]:
row2 = rows[1]
row2

In [ ]:
np.allclose(row2.sample.toArray(), sample2)

In [ ]:
sess.close()

# Convert rdd partitions -> tfrecord files

In [ ]:
base_dir = "data/tf"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")
for folder in [train_dir, val_dir]:
  if not os.path.exists(folder):
    os.mkdir(folder)
train_dir, val_dir

In [ ]:
train_df.repartition(3).foreachPartition(lambda rows: convert(rows, train_dir))
val_df.repartition(3).foreachPartition(lambda rows: convert(rows, val_dir))

In [ ]:
FOLDER = train_dir
[os.path.join(FOLDER, f) for f in os.listdir(FOLDER)]

In [ ]:
tf.train.shuffle_batch?

In [ ]:
tf.reset_default_graph()

# Hyperparams & Settings
folder=train_dir
min_after_dequeue = 1000
batch_size = 100  #64
capacity = min_after_dequeue + 4 * batch_size
num_threads = 2
num_epochs = 100
classes = 3
features = 65536
C = 1  # Number of input channels (dimensionality of input depth)
Hin = 256  # Input height
Win = 256  # Input width
Hf = 3  # conv filter height
Wf = 3  # conv filter width
Hfp = 2  # pool filter height
Wfp = 2  # pool filter width
stride = 1  # conv stride
pstride = 2  # pool stride
pad = 1  # For same dimensions, (Hf - stride) / 2
F1 = 32  # num conv filters in conv1
F2 = 32  # num conv filters in conv2
F3 = 32  # num conv filters in conv3
N1 = 512  # num nodes in affine1
lr = 1e-4 # learning rate

# Inputs
with tf.name_scope("input") as scope:
  # Input queues
  filenames = [os.path.join(folder, f) for f in os.listdir(folder)]
  filename_queue = tf.train.string_input_producer(filenames, num_epochs=num_epochs)
  index, slide_num, tumor_score, molecular_score, sample = read_and_decode(filename_queue)
  sample.set_shape([Hin*Win])
  x, y_ = tf.train.shuffle_batch([sample, tumor_score], batch_size=batch_size, capacity=capacity,
                                 min_after_dequeue=min_after_dequeue, num_threads=num_threads)
  
  #x = tf.placeholder(tf.float32, [None, features], name="x")
  x_image = tf.transpose(tf.reshape(x, [-1, C, Hin, Win]), perm=[0,2,3,1])  # shape (N,H,W,C)
  #y_ = tf.placeholder(tf.int64, [None, ], name="y_")
  y_one_hot = tf.one_hot(y_-1, classes)  # or use sparse cross entropy
  tf.summary.image("x", x_image)
  tf.summary.histogram("y", y_)

# Conv layer 1: conv1 -> relu1 -> pool1
with tf.name_scope("conv1") as scope:
  W = tf.Variable(tf.random_normal([Hf, Wf, C, F1]) * np.sqrt(2.0/(Hf*Wf*C)), name="W")
  b = tf.Variable(tf.zeros([F1]), name="b")
  conv = tf.nn.conv2d(x_image, W, [1,stride,stride,1], padding="SAME") + b
  relu = tf.nn.relu(conv)
  pool = tf.nn.max_pool(relu, ksize=[1,Hfp,Wfp,1], strides=[1,pstride,pstride,1], padding="SAME")
  tf.summary.image("conv1", tf.transpose(W, [3,0,1,2]), max_outputs=F1)  # transpose to [N,H,W,C]

# Conv layer 2: conv2 -> relu2 -> pool2
with tf.name_scope("conv2") as scope:
  W = tf.Variable(tf.random_normal([Hf, Wf, F1, F2]) * np.sqrt(2.0/(Hf*Wf*F1)), name="W")
  b = tf.Variable(tf.zeros([F2]), name="b")
  conv = tf.nn.conv2d(pool, W, [1,stride,stride,1], padding="SAME") + b
  relu = tf.nn.relu(conv)
  pool = tf.nn.max_pool(relu, ksize=[1,Hfp,Wfp,1], strides=[1,pstride,pstride,1], padding="SAME")

# Conv layer 3: conv3 -> relu3 -> pool3
with tf.name_scope("conv3") as scope:
  W = tf.Variable(tf.random_normal([Hf, Wf, F2, F3]) * np.sqrt(2.0/(Hf*Wf*F2)), name="W")
  b = tf.Variable(tf.zeros([F3]), name="b")
  conv = tf.nn.conv2d(pool, W, [1,stride,stride,1], padding="SAME") + b
  relu = tf.nn.relu(conv)
  pool = tf.nn.max_pool(relu, ksize=[1,Hfp,Wfp,1], strides=[1,pstride,pstride,1], padding="SAME")

# Affine layer 1:  affine1 -> relu1 -> dropout
with tf.name_scope("affine1") as scope:
  D = int(F3*(Hin/2**3)*(Win/2**3))
  W = tf.Variable(tf.random_normal([D,N1]) * np.sqrt(2.0/D), name="W")
  b = tf.Variable(tf.zeros([N1]), name="b")
  affine = tf.matmul(tf.reshape(pool, [-1,D]), W) + b
  relu = tf.nn.relu(affine)
  keep_prob = tf.placeholder(tf.float32, name="keep_prob")
  dropout = tf.nn.dropout(relu, keep_prob)

# Affine layer 2:  affine2 -> softmax
with tf.name_scope("affine2") as scope:
  W = tf.Variable(tf.random_normal([N1,classes]) * np.sqrt(2.0/N1), name="W")
  b = tf.Variable(tf.zeros([classes]), name="b")
  logits = tf.matmul(dropout, W) + b
  probs = tf.nn.softmax(logits)
  tf.summary.histogram("logits", logits)
  tf.summary.histogram("probs", probs)

# Loss
with tf.name_scope("loss") as scope:
  cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, y_one_hot))
  tf.summary.scalar("loss", cross_entropy_loss)

# Train
# train_step = tf.train.GradientDescentOptimizer(lr).minimize(cross_entropy)
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy_loss)

# Eval metrics
with tf.name_scope("eval") as scope:
  correct_pred = tf.equal(tf.argmax(logits,1), tf.argmax(y_one_hot,1))
  accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
  tf.summary.scalar("accuracy", accuracy)

In [ ]:
# Run `tensorboard --logdir=tf_logs --host=localhost --debug --reload_interval 5`
with tf.Session() as sess:
  # Summaries
  log_dir = "tf_logs"
  summary_op = tf.summary.merge_all()
  train_writer = tf.train.SummaryWriter(log_dir + "/train", sess.graph)
  val_writer = tf.train.SummaryWriter(log_dir + "/val")
  
  init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
  sess.run(init_op)
  
  # Start input queues
  coord = tf.train.Coordinator()
  threads = tf.train.start_queue_runners(sess=sess, coord=coord)
  
  # Data Gen
  #train_generator = gen_batch(train_df.rdd, 64)
  x_val, y_val = extract_data(val_df)
  
  # Train
  try:
    i = 0
    while not coord.should_stop():
      _ = sess.run([train_step], feed_dict={keep_prob:0.5})
      if i % 10 == 0:
        # train stats
        summary, train_acc = sess.run([summary_op, accuracy], feed_dict={keep_prob:0.5})
        train_writer.add_summary(summary, i)
        # val stats
        summary, val_acc = sess.run([summary_op, accuracy], feed_dict={x: x_val, y_:y_val, keep_prob:0.5})
        val_writer.add_summary(summary, i)
        #train_writer.flush()  # To force write -- probably slower (usually asynchronous)
        #val_writer.flush()  # To force write -- probably slower (usually asynchronous)
        print("Iter: {}, \t Train Accuracy: {:.4f}, \t Val Accuracy: {:.4f}".format(i, train_acc, val_acc))
      i += 1
  except tf.errors.OutOfRangeError:
    print("Done training!")
  finally:
    # Ask threads to stop when finished
    coord.request_stop()
  # Wait for all threads to finish  
  coord.join(threads)
  
train_writer.flush()  # Make sure everything is written before exiting
val_writer.flush()  # Make sure everything is written before exiting